In [ ]:
!pip3 install pycuda

     |████████████████████████████████| 1.6MB 5.1MB/s 
     |████████████████████████████████| 61kB 20.8MB/s 
     |████████████████████████████████| 471kB 43.3MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4536169 sha256=2adda0ff04b8b50a044c66ec231ac21fa4453e37e1f8d3b2da1c72780a90d654
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2019.1.1-py2.py3-none-any.whl size=58424 sha256=b15ee8836a38cfae617c0ce09b9ee95ab343ed038b7c049a143f4c3daa5cb077
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Created wheel for mako: filename=Mako-1.1.0-cp36-none-any.whl size=75363 sha256=e9ef874b61a73e399c62d44b924c962a9a065dfa3391239f171c8971256dab83
  Stored in directory: /root/.cache/pip/wheels/98/32/7b/a291926643fc1d1e02593e0d9e247c5a866a366b8343b7aa27
Successfully built pycuda pytools mako


In [ ]:
import math
import numpy as np
import pycuda.gpuarray as gpuarray
from pycuda.compiler import SourceModule
import cuda_utils as cu
import pycuda.autoinit

In [ ]:
module = SourceModule("""
texture<int, 3, cudaReadModeElementType> tex_3d;
__global__ void read_texture_3d(int nx, int ny, int nz, int *res){
    int x = threadIdx.x + blockDim.x * blockIdx.x;
    int y = threadIdx.y + blockDim.y * blockIdx.y;
    int z = threadIdx.z + blockDim.z * blockIdx.z;
    int ijk = nx * ny * z + nx * y + x;
    if (x < nx && y < ny && z < nz){
        res[ijk] = tex3D(tex_3d, x, y, z);
    }
}
""")

In [ ]:
read_tex_3d = module.get_function("read_texture_3d")

In [ ]:
num_components = np.int32(24)

In [ ]:
arr = np.arange(num_components, dtype=np.int32)

In [ ]:
np.random.shuffle(arr)

In [ ]:
num_x, num_y, num_z = np.int32(2), np.int32(3), np.int32(4)

In [ ]:
arr = arr.reshape(num_z, num_y, num_x)

In [ ]:
print(arr)

[[[ 5 20]
  [22 16]
  [15  4]]

 [[ 8  3]
  [12  7]
  [13 17]]

 [[ 6  2]
  [ 1 21]
  [18 14]]

 [[19 23]
  [11  0]
  [ 9 10]]]


In [ ]:
arr_gpu = gpuarray.to_gpu(arr)
res_gpu = gpuarray.zeros([num_z, num_y, num_x], dtype=np.int32)

In [ ]:
tex_3d = module.get_texref("tex_3d")

In [ ]:
cu.bind_array_to_texture3d(arr, tex_3d)

In [ ]:
threads_per_block = (6, 6, 6)
block_x = math.ceil(num_x / threads_per_block[0])
block_y = math.ceil(num_y / threads_per_block[1])
block_z = math.ceil(num_z / threads_per_block[2])
blocks_per_grid = (block_x, block_y, block_z)

In [ ]:
read_tex_3d(num_x, num_y, num_z, res_gpu, block=threads_per_block, grid=blocks_per_grid, texrefs=[tex_3d])

In [ ]:
res_gpu.get()

array([[[ 5, 20],
        [22, 16],
        [15,  4]],

       [[ 8,  3],
        [12,  7],
        [13, 17]],

       [[ 6,  2],
        [ 1, 21],
        [18, 14]],

       [[19, 23],
        [11,  0],
        [ 9, 10]]], dtype=int32)